# EVA8 Session 11 Assignment - Part 1
## BERT Custom Retraining

## Goals:
1. Collect custom dataset
2. Perform noisy word prediction(swap any word 15% of times from a sentence with any other random word, and then predict the correct word)
3. Share sample from dataset, training logs and 10 examples of input-output

## Import Libraries

In [1]:
from torch.utils.data import Dataset
from collections import Counter
import torch.nn.functional as F
from bs4 import BeautifulSoup
import torch.optim as optim
from os.path import exists
from tqdm import tqdm
import torch.nn as nn
import numpy as np
import random
import urllib
import torch
import math
import re

/opt/anaconda3/envs/PhotogradeML/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Define Transformer BERT Model (Encoder Only)

In [2]:
# =============================================================================
# Transformer
# =============================================================================
def attention(q, k, v, mask = None, dropout = None):
    scores = q.matmul(k.transpose(-2, -1))
    scores /= math.sqrt(q.shape[-1])
    
    #mask
    scores = scores if mask is None else scores.masked_fill(mask == 0, -1e3)
    
    scores = F.softmax(scores, dim = -1)
    scores = dropout(scores) if dropout is not None else scores
    output = scores.matmul(v)
    return output

class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, out_dim, dropout=0.1):
        super().__init__()
        
#        self.q_linear = nn.Linear(out_dim, out_dim)
#        self.k_linear = nn.Linear(out_dim, out_dim)
#        self.v_linear = nn.Linear(out_dim, out_dim)
        self.linear = nn.Linear(out_dim, out_dim*3)

        self.n_heads = n_heads
        self.out_dim = out_dim
        self.out_dim_per_head = out_dim // n_heads
        self.out = nn.Linear(out_dim, out_dim)
        self.dropout = nn.Dropout(dropout)
    
    def split_heads(self, t):
        return t.reshape(t.shape[0], -1, self.n_heads, self.out_dim_per_head)
    
    def forward(self, x, y=None, mask=None):
        #in decoder, y comes from encoder. In encoder, y=x
        y = x if y is None else y
        
        qkv = self.linear(x) # BS * SEQ_LEN * (3*EMBED_SIZE_L)
        q = qkv[:, :, :self.out_dim] # BS * SEQ_LEN * EMBED_SIZE_L
        k = qkv[:, :, self.out_dim:self.out_dim*2] # BS * SEQ_LEN * EMBED_SIZE_L
        v = qkv[:, :, self.out_dim*2:] # BS * SEQ_LEN * EMBED_SIZE_L
        
        #break into n_heads
        q, k, v = [self.split_heads(t) for t in (q,k,v)]  # BS * SEQ_LEN * HEAD * EMBED_SIZE_P_HEAD
        q, k, v = [t.transpose(1,2) for t in (q,k,v)]  # BS * HEAD * SEQ_LEN * EMBED_SIZE_P_HEAD
        
        #n_heads => attention => merge the heads => mix information
        scores = attention(q, k, v, mask, self.dropout) # BS * HEAD * SEQ_LEN * EMBED_SIZE_P_HEAD
        scores = scores.transpose(1,2).contiguous().view(scores.shape[0], -1, self.out_dim) # BS * SEQ_LEN * EMBED_SIZE_L
        out = self.out(scores)  # BS * SEQ_LEN * EMBED_SIZE
        
        return out

class FeedForward(nn.Module):
    def __init__(self, inp_dim, inner_dim, dropout=0.1):
        super().__init__()
        self.linear1 = nn.Linear(inp_dim, inner_dim)
        self.linear2 = nn.Linear(inner_dim, inp_dim)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        #inp => inner => relu => dropout => inner => inp
        return self.linear2(self.dropout(F.relu(self.linear1(x)))) 

class EncoderLayer(nn.Module):
    def __init__(self, n_heads, inner_transformer_size, inner_ff_size, dropout=0.1):
        super().__init__()
        self.mha = MultiHeadAttention(n_heads, inner_transformer_size, dropout)
        self.ff = FeedForward(inner_transformer_size, inner_ff_size, dropout)
        self.norm1 = nn.LayerNorm(inner_transformer_size)
        self.norm2 = nn.LayerNorm(inner_transformer_size)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
    
    def forward(self, x, mask=None):
        x2 = self.norm1(x)
        x = x + self.dropout1(self.mha(x2, mask=mask))
        x2 = self.norm2(x)
        x = x + self.dropout2(self.ff(x2))
        return x

class Transformer(nn.Module):
    def __init__(self, n_code, n_heads, embed_size, inner_ff_size, n_embeddings, seq_len, dropout=.1):
        super().__init__()
        
        #model input
        self.embeddings = nn.Embedding(n_embeddings, embed_size)
        self.pe = PositionalEmbedding(embed_size, seq_len)
        
        #backbone
        encoders = []
        for i in range(n_code):
            encoders += [EncoderLayer(n_heads, embed_size, inner_ff_size, dropout)]
        self.encoders = nn.ModuleList(encoders)
        
        #language model
        self.norm = nn.LayerNorm(embed_size)
        self.linear = nn.Linear(embed_size, n_embeddings, bias=False)
                
    
    def forward(self, x):
        x = self.embeddings(x)
        x = x + self.pe(x)
        for encoder in self.encoders:
            x = encoder(x)
        x = self.norm(x)
        x = self.linear(x)
        return x

# Positional Embedding
class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_seq_len = 80):
        super().__init__()
        self.d_model = d_model
        pe = torch.zeros(max_seq_len, d_model)
        pe.requires_grad = False
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        return self.pe[:,:x.size(1)] #x.size(1) = seq_len

## Dataset Creation/Handling Code

In [3]:
# =============================================================================
# Dataset
# =============================================================================
class SentencesDataset(Dataset):
    #Init dataset
    def __init__(self, sentences, vocab, seq_len):
        dataset = self
        
        dataset.sentences = sentences
        dataset.vocab = vocab + ['<ignore>', '<oov>', '<mask>']
        dataset.vocab = {e:i for i, e in enumerate(dataset.vocab)} 
        dataset.rvocab = {v:k for k,v in dataset.vocab.items()}
        dataset.seq_len = seq_len
        
        #special tags
        dataset.IGNORE_IDX = dataset.vocab['<ignore>'] #replacement tag for tokens to ignore
        dataset.OUT_OF_VOCAB_IDX = dataset.vocab['<oov>'] #replacement tag for unknown words
        dataset.MASK_IDX = dataset.vocab['<mask>'] #replacement tag for the masked word prediction task
    
    
    #fetch data
    def __getitem__(self, index, p_random_switch=0.15):
        dataset = self
        
        #while we don't have enough word to fill the sentence for a batch
        s = []
        while len(s) < dataset.seq_len:
            s.extend(dataset.get_sentence_idx(index % len(dataset)))
            index += 1
        
        #ensure that the sequence is of length seq_len
        s = s[:dataset.seq_len]
        [s.append(dataset.IGNORE_IDX) for i in range(dataset.seq_len - len(s))] #PAD ok
        
        #apply random mask
        random_index = random.randint(0, (len(dataset.vocab)-1))
        s = [(random_index, w) if random.random() < p_random_switch else (w, w) for w in s]
        
        return {'input': torch.Tensor([w[0] for w in s]).long(),
                'target': torch.Tensor([w[1] for w in s]).long()}

    #return length
    def __len__(self):
        return len(self.sentences)

    #get words id
    def get_sentence_idx(self, index):
        dataset = self
        s = dataset.sentences[index]
        s = [dataset.vocab[w] if w in dataset.vocab else dataset.OUT_OF_VOCAB_IDX for w in s] 
        return s
    


## Create Dataset

In [4]:
# Following Dataset Creation Code works only on Wikipedia, Use of other websites not tested

## Extracts Paragraph Text and Links from webpage link input
def extract_data(link, linkLimit = 2000, extractLinks = False):
    
    response = urllib.request.urlopen(link)

    html = response.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    
    pageText = ""
    
    for content in soup.find_all('p'):
        pageText += content.get_text()
    
    pageLinks = []
    count = 0
    if extractLinks:
        print("Processing page links....")
        for subLink in soup.find_all('a', href=True):
            if "/wiki/" in subLink['href']  and ":" not in subLink['href'] and "wikimedia" not in subLink['href']:
                pageLinks.append("https://en.wikipedia.org" + subLink['href'])
                count += 1
            if count == linkLimit: ## Limit dataset collection to 2000 links
                break
    
    return pageText, pageLinks
    
## Processes text to remove reference numbers, brackets and text within brackets
def process_text(text):
    
    processedText = ""
    for line in text.split("\n"):
        
        if ":" in line or len(line.strip()) <= 5: ## Ignores info captions or short lines
            continue
        else:
            line = re.sub("\(.*?\)", "", line) ## remove any text contained with and within brackets
            line = re.sub("\[.*?\]", "", line) ## remove reference numbers and square brackets
            line = re.sub("\"", "", line) ## remove quotes (can cause errors)
            processedText += line + "\n"
    
    return processedText

## Main function that parses text from all wiki links found within startPage (including the text on startPage)
def collect_dataset(startPage):
    
    ## Get Text and Links from start_page
    startPageText, startPageLinks = extract_data(startPage, extractLinks = True)
    
    allText = ""
    processedText = process_text(startPageText)
    #print(processedText)
    allText += processedText
    #print(allText)
    
    pbar = tqdm(startPageLinks)
    for i, link in enumerate(pbar):
        pageText, _ = extract_data(link)
        processedPageText = process_text(pageText)
        allText += processedText
    
    return allText
    
    

## Collect Text Data for Dataset

In [5]:
## Creates dataset from Code defined in previous block
## Text is scraped from the link input as well as all wiki links found within the linked page
datasetText = collect_dataset("https://en.wikipedia.org/wiki/India")

Processing page links....


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [25:48<00:00,  1.29it/s]


In [6]:
## Save dataset to txt file
with open('BERT_AssignmentDataset.txt', 'w') as f:
    f.write(datasetText)

## Initialization & Dataset Tokenization

In [7]:
# =============================================================================
# Methods / Class
# =============================================================================
def get_batch(loader, loader_iter):
    try:
        batch = next(loader_iter)
    except StopIteration:
        loader_iter = iter(loader)
        batch = next(loader_iter)
    return batch, loader_iter

# =============================================================================
# #Init
# =============================================================================
print('initializing..')
batch_size = 1024
seq_len = 20
embed_size = 128
inner_ff_size = embed_size * 4
n_heads = 8
n_code = 8
n_vocab = 40000
dropout = 0.1
# n_workers = 12

#optimizer
optim_kwargs = {'lr':1e-4, 'weight_decay':1e-4, 'betas':(.9,.999)}

# =============================================================================
# Input
# =============================================================================
#1) load text
print('loading text...')
pth = 'BERT_AssignmentDataset.txt'
sentences = open(pth).read().lower().split('\n')
#sentences = datasetText.lower().split("\n")

#2) tokenize sentences (can be done during training, you can also use spacy udpipe)
print('tokenizing sentences...')
special_chars = ',?;.:/*!+-()[]{}"\'&'
sentences = [re.sub(f'[{re.escape(special_chars)}]', ' \g<0> ', s).split(' ') for s in sentences]
sentences = [[w for w in s if len(w)] for s in sentences]

#3) create vocab if not already created
print('creating/loading vocab...')
pth = 'vocab_bert_assigment11.txt'
if not exists(pth):
    words = [w for s in sentences for w in s]
    vocab = Counter(words).most_common(n_vocab) #keep the N most frequent words
    vocab = [w[0] for w in vocab]
    open(pth, 'w+').write('\n'.join(vocab))
else:
    vocab = open(pth).read().split('\n')

#4) create dataset
print('creating dataset...')
dataset = SentencesDataset(sentences, vocab, seq_len)
# kwargs = {'num_workers':n_workers, 'shuffle':True,  'drop_last':True, 'pin_memory':True, 'batch_size':batch_size}
kwargs = {'shuffle':True,  'drop_last':True, 'pin_memory':True, 'batch_size':batch_size}
data_loader = torch.utils.data.DataLoader(dataset, **kwargs)

initializing..
loading text...
tokenizing sentences...
creating/loading vocab...
creating dataset...


## Initialize Device

In [8]:
## CPU Device
DATA_DIR='./data'
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("DEVICE:", DEVICE)

DEVICE: cpu


In [9]:
## Apple Silicon Metal Performance Shader
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")
    device = torch.device("cpu")

else:
    device = torch.device("mps")
    
print("device:", device)

device: mps


## Model Initialization & Training

In [10]:


# =============================================================================
# Model
# =============================================================================
#init model
print('initializing model...')
model = Transformer(n_code, n_heads, embed_size, inner_ff_size, len(dataset.vocab), seq_len, dropout)
#model = model.cuda()
model.to(DEVICE)

# =============================================================================
# Optimizer
# =============================================================================
print('initializing optimizer and loss...')
optimizer = optim.Adam(model.parameters(), **optim_kwargs)
loss_model = nn.CrossEntropyLoss(ignore_index=dataset.IGNORE_IDX)

# =============================================================================
# Train
# =============================================================================
print('training...')
print_each = 10
model.train()
batch_iter = iter(data_loader)
n_iteration = 1000
for it in range(n_iteration):
    
    #get batch
    batch, batch_iter = get_batch(data_loader, batch_iter)
    
    #infer
    masked_input = batch['input']
    masked_target = batch['target']
    
    #masked_input = masked_input.cuda(non_blocking=True)
    #masked_target = masked_target.cuda(non_blocking=True)
    masked_input.to(DEVICE)
    masked_target.to(DEVICE)
    
    output = model(masked_input)
    
    #compute the cross entropy loss - original code
    output_v = output.view(-1,output.shape[-1])
    target_v = masked_target.view(-1,1).squeeze()
    loss = loss_model(output_v, target_v)
    
    #compute gradients
    loss.backward()
    
    #apply gradients
    optimizer.step()
    
    #print step
    if it % print_each == 0:
        print('it:', it, 
              ' | loss', np.round(loss.item(),2),
              ' | Δw:', round(model.embeddings.weight.grad.abs().sum().item(),3))
    
    #reset gradients
    optimizer.zero_grad()

initializing model...
initializing optimizer and loss...
training...
it: 0  | loss 8.04  | Δw: 3.759
it: 10  | loss 7.19  | Δw: 3.045
it: 20  | loss 6.64  | Δw: 3.123
it: 30  | loss 6.19  | Δw: 3.218
it: 40  | loss 5.78  | Δw: 3.091
it: 50  | loss 5.43  | Δw: 2.951
it: 60  | loss 5.12  | Δw: 2.848
it: 70  | loss 4.85  | Δw: 2.753
it: 80  | loss 4.6  | Δw: 2.644
it: 90  | loss 4.42  | Δw: 2.611
it: 100  | loss 4.21  | Δw: 2.52
it: 110  | loss 4.03  | Δw: 2.474
it: 120  | loss 3.86  | Δw: 2.452
it: 130  | loss 3.67  | Δw: 2.364
it: 140  | loss 3.55  | Δw: 2.343
it: 150  | loss 3.39  | Δw: 2.284
it: 160  | loss 3.23  | Δw: 2.236
it: 170  | loss 3.14  | Δw: 2.199
it: 180  | loss 2.98  | Δw: 2.152
it: 190  | loss 2.9  | Δw: 2.126
it: 200  | loss 2.81  | Δw: 2.087
it: 210  | loss 2.7  | Δw: 2.037
it: 220  | loss 2.58  | Δw: 1.989
it: 230  | loss 2.49  | Δw: 1.951
it: 240  | loss 2.38  | Δw: 1.922
it: 250  | loss 2.31  | Δw: 1.878
it: 260  | loss 2.2  | Δw: 1.823
it: 270  | loss 2.15  | Δw: 1

## Sample Inputs & Outputs

### Generate Outputs

In [11]:
batch, batch_iter = get_batch(data_loader, batch_iter)

masked_input = batch['input']
masked_target = batch['target']

masked_input.to(DEVICE)

output = model(masked_input)
print(output.shape)

torch.Size([1024, 20, 2576])


### Convert Output to Token Indices

In [12]:
m = nn.Softmax(dim = 2)
probOutput = m(output)
idxOutput = torch.argmax(probOutput, dim = 2)
print(idxOutput.shape)

torch.Size([1024, 20])


### Print Input/Output

In [13]:
 
for i in range(10):
    inp = masked_input[i]
    outp = idxOutput[i]
    
    print(f"------------Example {i}---------------")
    inp_text = [dataset.rvocab[int(idx)] for idx in inp]
    print("Input: " + ' '.join(inp_text))
    outp_text = [dataset.rvocab[int(idx)] for idx in outp]
    print("Output: " + ' '.join(outp_text))

------------Example 0---------------
Input: government general information modern iconic arrived on the iconic iconic iconic iconic no later than 55 , 000 years ago
Output: government general information modern humans arrived on the humans humans humans humans no later than 55 , 000 years ago
------------Example 1---------------
Input: many indian species have classical from classical of gondwana , the southern supercontinent from which india separated more than classical
Output: many indian species have indian from , of gondwana , the southern supercontinent from which india separated more than indian
------------Example 2---------------
Input: elephanta caves , becoming becoming bust of shiva , 18 becoming tall , becoming .  550 becoming bronze of shiva
Output: elephanta caves , and and bust of shiva , 18 , tall , and .  550 and bronze of shiva
------------Example 3---------------
Input: india mamuni substantially mamuni its rate of poverty , though at the cost of increasing economic

### Single Input/Output

In [14]:
model_inp = masked_input[2]
model_out = model(model_inp)
print(model_out.shape)
mid = m(model_out)
idxOut_model = torch.argmax(mid, dim = 2)
print(idxOut_model.shape)
print(idxOut_model)

inp_text = [dataset.rvocab[int(idx)] for idx in model_inp]
print("Input: " + ' '.join(inp_text))
outp_text = [dataset.rvocab[int(idx2)] for idx2 in idxOut_model[0]]
print("Output: " + ' '.join(outp_text))

torch.Size([1, 20, 2576])
torch.Size([1, 20])
tensor([[ 869,  511,    0,    1,    3, 2113,    4,  366,    0,  881,    3, 2115,
            0,    3,    2, 2116,    3,  871,    4,  366]])
Input: elephanta caves , becoming becoming bust of shiva , 18 becoming tall , becoming .  550 becoming bronze of shiva
Output: elephanta caves , the and bust of shiva , 18 and tall , and .  550 and bronze of shiva
